In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('data/Merge.csv')

In [3]:
df.Resolution.unique()

array(['10MIN', '1H', 'D'], dtype=object)

In [5]:
def sub_2h_resampling(df, freq):
    filtered_df = df[df['Resolution'] == freq].copy()
    filtered_df['Datetime'] = pd.to_datetime(filtered_df['Datetime'])
    resampled_df = filtered_df.resample('2H', on='Datetime', closed='left', offset='1h').mean().dropna().rename(columns={'Price': f'Price_{freq}'})
    return resampled_df

In [6]:
def trading_hours_index(date_start, date_end):
    """
    The daily granularity data becomes a bit more complex as we need to resample it to 07:00 - 17:00
    For this wI created a custom calendar/index for between 07:00 and 17:00 for the defined dates
    """
    index_date = pd.date_range(date_start, date_end)
    index_date = pd.Series(index_date)
    index_time = pd.date_range('07:00:00', '15:00:00', freq='2H')
    index_time = pd.Series(index_time.time)

    index = index_date.apply(
        lambda d: index_time.apply(
            lambda t: datetime.combine(d, t)
            )
        ).unstack().sort_values().reset_index(drop=True)
    return index

In [7]:
def daily_resampling(df, start_date, end_date):
    """
    I will use the re index function, but for this to work a value in the old index must match a value in the 
    new index (where the day starts at 07:00)
    For this reason I just add 7 hours to the original Datetime in the Daily granularity -> 2021-11-01 07:00:00
    """
    filtered_df = df[df['Resolution'] == 'D'].copy()
    filtered_df['Datetime'] = pd.to_datetime(filtered_df['Datetime'])
    filtered_df['Datetime'] = filtered_df['Datetime'] + pd.to_timedelta(7, unit='h')
    
    index = trading_hours_index(start_date, end_date)
    
    resampled_daily = filtered_df[['Datetime','Price']].set_index('Datetime').reindex(index=index, method = 'ffill').rename(columns={'Price': 'Price_daily'})
    resampled_daily.index.name = 'Datetime'
    
    return resampled_daily

In [8]:
resampled_10min = sub_2h_resampling(df, '10MIN')
resampled_1h = sub_2h_resampling(df, '1H')

resampled_daily = daily_resampling(df, '2021-11-01', '2021-11-30')

In [12]:
merged_df = resampled_10min.merge(resampled_1h, how='left', on ='Datetime').merge(resampled_daily, how='left', on = 'Datetime')

In [13]:
merged_df.to_csv('outputs/q1.csv')

In [14]:
merged_df

,Price_10MIN,Price_1H,Price_daily
Datetime,,,
2021-11-01 07:00:00,70.857500,71.2625,66.1
2021-11-01 09:00:00,71.566667,71.0625,66.1
2021-11-01 11:00:00,68.812500,67.5750,66.1
2021-11-01 13:00:00,63.250000,63.7000,66.1
2021-11-01 15:00:00,65.589583,65.8750,66.1
...,...,...,...
2021-11-30 07:00:00,97.335714,97.8750,91.7
2021-11-30 09:00:00,98.860417,98.6250,91.7
2021-11-30 11:00:00,96.662500,96.3625,91.7
